In [63]:
import pandas as pd
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
sw=stopwords.words('english')
ps=PorterStemmer()
import json 

In [88]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
;WITH  CURR AS (
	SELECT ID,QGROUP,FK_FormPage,QPK,MAX_Q_PK_STAG,QOFFSET,MAX_QG_PK_STAG,QGOFFSET,MAX_FK_Form_STAG,FK_Form_OFFSET 
	FROM vw_MetricsCompositeKeys WHERE PK_Form = '2020-A-SAOP'  
), NEXT AS ( 
	SELECT 
		 CURR.ID  
		,CURR.QGOFFSET 
		,CURR.QPK CURR_QPK 
		,CURR.MAX_Q_PK_STAG + CURR.QOFFSET NEXT_QPK  
		,CURR.QGROUP CURR_QGROUP
		,CURR.MAX_QG_PK_STAG + CURR.QGOFFSET NEXT_QGROUP  
		,CURR.FK_FormPage CURR_FK_FormPage
		,CURR.MAX_FK_Form_STAG + CURR.FK_Form_OFFSET NEXT_FK_FormPage
	FROM CURR  
) 
SELECT * FROM NEXT
""" 
df = pd.read_sql(query,con=conn)   
df.to_excel(r'C:\_som\_src\_compile\saop\SAOP.xlsx','SAOP') 
df 

,ID,QGOFFSET,CURR_QPK,NEXT_QPK,CURR_QGROUP,NEXT_QGROUP,CURR_FK_FormPage,NEXT_FK_FormPage
0,1-0-,0,21200,23838,2150,2627,2090,2527
1,1-1-1a,0,21201,23839,2150,2627,2090,2527
2,1-2-1b,0,21202,23840,2150,2627,2090,2527
3,1-3-1c,0,21203,23841,2150,2627,2090,2527
4,1-4-1d,0,21204,23842,2150,2627,2090,2527
...,...,...,...,...,...,...,...,...
92,17-7-14g,16,21402,24040,2166,2643,2106,2543
93,17-9-14h,16,21403,24041,2166,2643,2106,2543
94,17-10-14i,16,21404,24042,2166,2643,2106,2543
95,17-11-14j,16,21405,24043,2166,2643,2106,2543


In [101]:
QGOFFSET = -1
QPK = 25000
QGR = 3000
d = {}
for i,r in df.iterrows():  
    if str(r.CURR_QGROUP) not in d.keys():
        QGR+=1
        d[str(r.CURR_QGROUP)]= str(QGR)     
for i,r in df.iterrows():      
    if QGOFFSET != r.QGOFFSET:
        QGOFFSET = r.QGOFFSET
        QPK += 25
    QPK+=1
    d[str(r.CURR_QPK)]= str(QPK)
 
with open("C:\_som\_src\_compile\saop\gapkeyval.json", "w") as f: 
    json.dump(d, f, indent=4)
    #f.write(json_string)


In [60]:
conn.close()

In [99]:
t = """
1234 Languages technologies platforms  is your web don't we have web driver 
"""
def encode(t):
    t = ' '.join([w for w in t.split(' ') if w not in sw])
    t = ' '.join([ps.stem(w) for w in t.split(' ')])
    t = t.upper()
    t = re.sub('[^A-Z0-9]', '', t )   
    # t = re.sub('[AEIOUY]', '', t)  
    return t
encode(t)

'1234LANGUAGTECHNOLOGPLATFORMWEBWEBDRIVER'